In [15]:
import numpy as np
import skimage as sk
import scipy as sc
import pyswarm

In [3]:
calc = sk.io.imread("calc.jpg")

In [16]:
def clustering_cost2(m, X, clusters):
    """
    Calculates the sum of within-cluster distances (WCD) for a given clustering configuration.

    Args:
        m (numpy.ndarray): A 3x6 matrix representing the cluster centers.
        X (numpy.ndarray): A data matrix where each row represents a data point.

    Returns:
        float: The total within-cluster distance (WCD).
    """

    # Ensure correct dimensions for m
    if m.shape != (3, 6):
        raise ValueError("Cluster center matrix (m) must have dimensions (3, 6).")

    # Reshape m into a cluster center matrix (4 clusters, 3 features)
    cluster_centers = m.reshape(3, clusters).T  # Transpose for column-wise centers

    # Calculate pairwise distances between data points and cluster centers
    distances = np.linalg.norm(X - cluster_centers[:, None, :], axis=2)  # Efficient broadcasting

    # Assign data points to closest clusters and find minimum distances
    closest_distances, cluster_assignments = np.min(distances, axis=0), np.argmin(distances, axis=0)

    # Calculate the sum of within-cluster distances (WCD)
    wcd = np.sum(closest_distances)

    return wcd

In [5]:
rows, cols, channels = calc.shape
rChannel = calc[..., 0]
gChannel = calc[..., 1]
bChannel = calc[..., 2]

In [6]:
x1 = (rChannel - np.min(rChannel.ravel())) / (np.max(rChannel.ravel()) - np.min(rChannel.ravel()))
x2 = (gChannel - np.min(gChannel.ravel())) / (np.max(gChannel.ravel()) - np.min(rChannel.ravel()))
x3 = (bChannel - np.min(bChannel.ravel())) / (np.max(bChannel.ravel()) - np.min(rChannel.ravel()))
x = np.array([x1.ravel(), x2.ravel(), x3.ravel()]).T
clusters = 6

In [17]:
varSize = np.array([clusters, x.shape[1]])
nVar = np.prod(varSize)
varMin = np.repeat(np.min(x, 0), clusters)
varMax = np.repeat(np.max(x, 0), clusters)

options = {'swarmsize': 100, 'maxiter': 20, 'omega': 0.5, 'c1': 1.5, 'c2': 2.0}
result = pyswarm.pso(clustering_cost2, options, lb=varMin, ub=varMax)